In [1]:
def define_searchspace(trial):
    model_type = trial.suggest_categorical('model', ['fsr_model.LSTM', 'fsr_model.CNN_LSTM', 'fsr_model.ANN'])
    if model_type == 'fsr_model.LSTM':
        trial.suggest_int('model_args/input_size', 6, 6)
        trial.suggest_int('model_args/output_size', 6, 6)
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model_type == 'fsr_model.CNN_LSTM':
        trial.suggest_int('model_args/input_size', 6, 6)
        trial.suggest_int('model_args/output_size', 6, 6)
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model_type == 'fsr_model.ANN':
        trial.suggest_int('model_args/input_size', 6, 6)
        trial.suggest_int('model_args/output_size', 6, 6)
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    trial.suggest_categorical('scaler', [
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
import datasource
from trainable import Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_parameters(Trainable, data=datasource.get_data()),
    tune_config=ray.tune.TuneConfig(
        num_samples=-1,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ),
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ),
) 
results = tuner.fit()

[I 2023-07-01 21:59:48,590] A new study created in memory with name: optuna
2023-07-01 21:59:50,814	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-01 21:59:52,222	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-01 21:59:52,268	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
Trainable_04647381,2023-07-01_22-00-56,True,DESKTOP-0P789CI,1,1.29847,1.11012e+15,172.26.215.93,2736,3.2987,1.19834,1.19834,1.19834,1688216456,1,04647381
Trainable_0cc4113a,2023-07-01_22-01-51,False,DESKTOP-0P789CI,21,0.0519432,3.32656e+13,172.26.215.93,1783,0.0957884,89.9398,5.10867,89.9398,1688216511,21,0cc4113a
Trainable_1bb49743,2023-07-01_22-00-27,True,DESKTOP-0P789CI,2,0.173485,3.14276e+14,172.26.215.93,2072,0.224151,1.81215,0.734499,1.81215,1688216427,2,1bb49743
Trainable_33ee8e2a,2023-07-01_22-00-57,True,DESKTOP-0P789CI,1,1.60251,1.26477e+14,172.26.215.93,2520,4.70597,13.092,13.092,13.092,1688216457,1,33ee8e2a
Trainable_3821056a,2023-07-01_22-01-20,True,DESKTOP-0P789CI,2,0.262059,6.21255e+14,172.26.215.93,3192,0.307477,2.05256,0.783925,2.05256,1688216480,2,3821056a
Trainable_4f7d0700,2023-07-01_22-01-52,False,DESKTOP-0P789CI,6,0.0755735,7.32722e+13,172.26.215.93,3661,0.133817,8.82443,1.93273,8.82443,1688216512,6,4f7d0700
Trainable_705aef17,2023-07-01_22-00-40,True,DESKTOP-0P789CI,2,0.184866,3.38038e+14,172.26.215.93,2310,0.226589,4.19977,1.96308,4.19977,1688216440,2,705aef17
Trainable_780a3455,2023-07-01_22-00-09,True,DESKTOP-0P789CI,1,1.65917,5.45146e+14,172.26.215.93,1585,4.53129,1.01357,1.01357,1.01357,1688216409,1,780a3455
Trainable_9677c295,2023-07-01_22-01-33,True,DESKTOP-0P789CI,2,0.621112,1.45401,172.26.215.93,3431,1.03601,2.86705,1.36489,2.86705,1688216493,2,9677c295
Trainable_bc8b1529,2023-07-01_22-01-50,False,DESKTOP-0P789CI,41,1.28433,1.1552e+15,172.26.215.93,1372,3.27737,98.8864,2.95259,98.8864,1688216510,41,bc8b1529


(_WandbLoggingActor pid=1433) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=1433) wandb: - Waiting for wandb.init()...
(_WandbLoggingActor pid=1433) wandb: \ Waiting for wandb.init()...
(_WandbLoggingActor pid=1433) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=1433) wandb: Run data is saved locally in /home/seokj/ray_results/Trainable_2023-07-01_21-59-48/Trainable_bc8b1529_1_criterion=torch_nn_MSELoss,model=fsr_model_CNN_LSTM,cnn_hidden_size=8,cnn_num_layer=3,input_size=6,lstm_hidde_2023-07-01_21-59-52/wandb/run-20230701_220003-bc8b1529
(_WandbLoggingActor pid=1433) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=1433) wandb: Syncing run Trainable_bc8b1529
(_WandbLoggingActor pid=1433) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=1433) wandb: 🚀 View run at https://wandb.ai/seokjin/FSR-prediction/runs/bc8b1529
(_WandbLoggingActor pi

(_WandbLoggingActor pid=3808) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=3808) wandb: Run data is saved locally in /home/seokj/ray_results/Trainable_2023-07-01_21-59-48/Trainable_4f7d0700_12_criterion=torch_nn_MSELoss,model=fsr_model_ANN,hidden_size=32,input_size=6,num_layer=8,output_size=6,optimiz_2023-07-01_22-01-28/wandb/run-20230701_220148-4f7d0700
(_WandbLoggingActor pid=3808) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=3808) wandb: Syncing run Trainable_4f7d0700
(_WandbLoggingActor pid=3808) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=3808) wandb: 🚀 View run at https://wandb.ai/seokjin/FSR-prediction/runs/4f7d0700
